In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scripts.data_processing import adjust_sales, product_sales_difference

In [2]:
pd.set_option('display.float_format', '{:.2f}'.format)
#%config InlineBackend.figure_format = 'svg' 

path_2022 = '../data/2022/'
path = '../data/'

FONT_TITLE = 25
FONT_LABEL = 15

In [3]:
def product_sales_difference_m(df, id_shop1, id_shop2, sum_sale=5000):
    '''
    Функци принимает на вход датафрейм, и id двух магазинов.
    
    Рассчитывает какие товары были проданны в первом магазине и 
    не проданы во втором.
    
    sum_sale = минимальная сумма выручки, которая береться у 
    первого магазина для отбора проданных товаров
    
    Возвращает датафрейм:
    Колонки: Количество, Сумма, Наименование, Подгруппа, Группа
    Индексы строк - id товаров
    
    Для наименования товара закачиваем датасет с ними
    inf_tovar.csv
    '''
    
    df_shop_1 = df[df['id_tt_cl'] == id_shop1]
    df_shop_2 = df[df['id_tt_cl'] == id_shop2]
    
    name_tov = pd.read_csv('../data/inf_tovar.csv', sep=';')
    #name_tov = name_tov.rename(columns={'Unnamed: 0': 'id_tov_cl'})
    
    gr_shop_1 = df_shop_1.groupby('id_tov_cl')[['Quantity', 'BaseSum']].sum().sort_values('BaseSum', ascending=False)
    gr_shop_2 = df_shop_2.groupby('id_tov_cl')[['Quantity', 'BaseSum']].sum().sort_values('BaseSum', ascending=False)

        
    id_tov_set1 = set(gr_shop_1[gr_shop_1['BaseSum'] > sum_sale].index)
    id_tov_set2 = set(gr_shop_2.index)
    diff_id_tov = list(id_tov_set1 - id_tov_set2)
    
    dict_tov_diff = {}
    i=0
    for el in diff_id_tov:
        dict_tov_diff[i] = {
            'id_tt_cl': id_shop1,
            'id_tov_cl': el,
            'Quantity': int(gr_shop_1['Quantity'][el]),
            'BaseSum': int(gr_shop_1['BaseSum'][el])
        }
        i+=1
    
    df_tov_diff = pd.DataFrame(dict_tov_diff).T
    
    if df_tov_diff.shape != (0, 0):
        df_tov_diff = df_tov_diff.merge(name_tov, how='left', on='id_tov_cl')
    
    return df_tov_diff

In [4]:
def group_tovar(df_shop):
    '''
    Функция группирует проданные товары по всем магазинам
    Принимает на вход датафрейм и на выход выдает датафрейм 
    с расчитанными значениями медианой по количеству и сумме продаж,
    подсчету количества магазинов, в которых в текущем месяце были реализованные данные товары
    и список этих магазинов
    '''
    
    
    df_shop['кол/площ'] = df_shop['Quantity']/df_shop['ploshad']
    df_shop['кол/народ'] = df_shop['Quantity']/df_shop['population']
    df_shop['сумма/площ'] = df_shop['BaseSum']/df_shop['ploshad']
    df_shop['сумма/народ'] = df_shop['BaseSum']/df_shop['population']
    
    df_shop = df_shop.groupby('id_tov_cl', as_index=False).agg({
                            'Quantity': 'median',
                            'BaseSum': 'median', 
                            'кол/площ': 'median',
                            'кол/народ': 'median',
                            'сумма/площ': 'median',
                            'сумма/народ': 'median',
                            'id_tt_cl': pd.Series.to_list,
                            'name': 'last',
                            'name_gr2': 'last',
                            'name_gr': 'last'})\
                     .rename(columns={'Quantity': 'медиана кол', 'BaseSum': 'медиана сумма', 'id_tt_cl': 'список магазинов'})\
                     .sort_values(by=['name_gr', 'name_gr2', 'name'])
    df_shop['кол маг'] = df_shop['список магазинов'].apply(lambda s: len(s))
    
    return df_shop

In [5]:
df_06_svao = pd.read_csv(path_2022+'svao_6_june.csv', sep=';')
svod_shop = pd.read_csv(path_2022+'svod_shop.csv', sep=';')
svod_shop = svod_shop.rename(columns={'Unnamed: 0':'id_tt_cl'})
name_tov = pd.read_csv('../data/inf_tovar.csv', sep=';')

In [6]:
svod_shop[svod_shop['id_tt_cl']==11127]

,id_tt_cl,population,count_shops,shop_list,adress,Shirota,Dolgota,ploshad,Район,shtat,...,5_mean,5_std,6_sum,6_mean,6_std,start_2,прирост_сред_3,прирост_сред_4,прирост_сред_5,прирост_сред_6
1,11127,2418.00,4.00,"Перекрёсток,Перекрёсток,Продуктовый магазин,Фе...","Москва ул. Пришвина, д. 26",55.88,37.60,105.80,Бибирево,4,...,274435.44,33040.23,7232107.73,267855.84,33553.55,0,4.12,-0.08,-5.97,-2.40


In [7]:
shop_list = list(df_06_svao['id_tt_cl'].unique())
# population = svod_shop[svod_shop['id_tt_cl']==11127]['population'].values[0]
# ploshad = svod_shop[svod_shop['id_tt_cl']==11127]['ploshad'].values[0]

In [9]:
# По формату магазинов ВкусВилл
for shop2 in [11127, 14258, 11178, 11357, 15714, 11732, 11956, 11271, 11591, 13968, 12046]:
    print(f'Обработка магазина {shop2}')
    
    df_shop = pd.DataFrame()
    
    for shop1 in shop_list:
        df_shop_tmp = product_sales_difference_m(df_06_svao, shop1, shop2, sum_sale=0)
        df_shop = pd.concat([df_shop, df_shop_tmp], axis=0, ignore_index=True)
    
    df_shop = df_shop.merge(svod_shop[['id_tt_cl', 'ploshad', 'population']], how='left', on='id_tt_cl') 
    
    df_shop = df_shop.sort_values(by=['name_gr', 'name_gr2', 'name'])    
    #df_shop.to_csv(path_2022+f'/tov_set/{shop2}_6_июнь_full.csv', sep=';', index=False)
    
    gr_shop = group_tovar(df_shop)
    
    population = int(svod_shop[svod_shop['id_tt_cl']==shop2]['population'].values[0])
    ploshad = svod_shop[svod_shop['id_tt_cl']==shop2]['ploshad'].values[0]
    
    gr_shop['кол/площ'] = gr_shop['кол/площ'] * ploshad
    gr_shop['кол/народ'] = gr_shop['кол/народ'] * population
    gr_shop['сумма/площ'] = gr_shop['сумма/площ'] * ploshad
    gr_shop['сумма/народ'] = gr_shop['сумма/народ'] * population
    
    #------------
    list_fname_2022 = ['svao_3_march.csv', 'svao_4_april.csv', 'svao_5_may.csv'] 
    list_month = ['март', 'апр', 'май']

    df_prod_2022 = pd.DataFrame(gr_shop['id_tov_cl'].to_list(), columns=['id_tov_cl'])

    for i in range(len(list_fname_2022)):
        print(i)
        df = pd.read_csv(path_2022+list_fname_2022[i], sep=';')

        df['Quantity'] = df['Quantity'] * df['znak']
        df['BaseSum'] = df['BaseSum'] * df['znak']

        df = df[df['id_tt_cl'] == shop2].groupby(['id_tov_cl'], as_index=False)\
             .agg({'Quantity': 'sum', 'BaseSum':'sum'})\
             .rename(columns={'Quantity':f'Кол_{list_month[i]}', 'BaseSum':f'Сумма_{list_month[i]}'})

        df_prod_2022 = df_prod_2022.merge(df, how='outer', on='id_tov_cl')

    df_prod_2022 = df_prod_2022.fillna(0)

    df_prod_2022['Кол_345'] = (df_prod_2022['Кол_март'] + df_prod_2022['Кол_апр'] + df_prod_2022['Кол_май'])

    df_prod_2022['Сумма_345'] = (df_prod_2022['Сумма_март'] + df_prod_2022['Сумма_апр'] + df_prod_2022['Сумма_май'])
    
    #---------------------
    
    df_finish = df_prod_2022.merge(gr_shop, how='right', on='id_tov_cl')
    
    # Список групп товаров в которых продано больльше 2 наименований 
    gr_tov = list(df_06_svao[df_06_svao['id_tt_cl']==shop2].merge(name_tov, how='left', on='id_tov_cl')\
                 .groupby('name_gr', as_index=False).agg({'name': pd.Series.nunique}).query('name > 2')['name_gr'])
    # отбор только товаров из этих групп
    df_finish = df_finish[df_finish['name_gr'].isin(gr_tov)]
    
    
    # Сохранение в файл. Выстраиваем колонки в нужном порядке
    col = ['id_tov_cl', 'Кол_март', 'Сумма_март',
       'Кол_апр', 'Сумма_апр', 'Кол_май', 'Сумма_май', 'Кол_345', 'Сумма_345',
       'медиана кол', 'медиана сумма', 'кол/площ', 'кол/народ', 'сумма/площ',
       'сумма/народ', 'кол маг', 'список магазинов', 'name', 'name_gr2', 'name_gr',
       ]
    ploshad = int(np.round(ploshad, 0))
    df_finish[col].to_csv(path_2022+f'отбор/{shop2}_s{ploshad}_n{population}_к_отбору.csv', sep=';', index=False)

Обработка магазина 11127
0
1
2
Обработка магазина 14258
0
1
2
Обработка магазина 11178
0
1
2
Обработка магазина 11357
0
1
2
Обработка магазина 15714
0
1
2
Обработка магазина 11732
0
1
2
Обработка магазина 11956
0
1
2
Обработка магазина 11271
0
1
2
Обработка магазина 11591
0
1
2
Обработка магазина 13968
0
1
2
Обработка магазина 12046
0
1
2


In [16]:
gr_shop[:5]

,id_tov_cl,медиана кол,медиана сумма,кол/площ,кол/народ,сумма/площ,сумма/народ,список магазинов,name,name_gr2,name_gr,кол маг
1699,58738,1.50,313.00,0.44,0.76,92.24,158.09,"[15123, 14330]",Бешбармак с мясом говядины Айс,Вторые блюда мясо айс,Айс ВкусВилл,2
646,36785,8.00,1576.00,2.30,4.07,454.04,801.17,"[15123, 14330]",Биточки мясные с картофелем и грибами Айс,Вторые блюда мясо айс,Айс ВкусВилл,2
431,30513,1.00,168.00,0.38,0.35,64.63,59.29,[11439],Говядина в соусе с черносливом и гречкой Айс,Вторые блюда мясо айс,Айс ВкусВилл,1
349,29108,3.00,586.00,1.15,1.16,225.45,227.01,"[11701, 15123, 14330, 11439, 11512]",Гречка с говядиной в сливочном соусе Айс,Вторые блюда мясо айс,Айс ВкусВилл,5
1452,54599,3.50,694.00,1.04,1.77,205.33,350.12,"[15123, 14330]",Котлета с паприкой и пюре из краснокочанной ка...,Вторые блюда мясо айс,Айс ВкусВилл,2


In [17]:
list_fname_2022 = ['svao_3_march.csv', 'svao_4_april.csv', 'svao_5_may.csv'] 
list_month = ['март', 'апр', 'май']

df_prod_2022 = pd.DataFrame(gr_shop['id_tov_cl'].to_list(), columns=['id_tov_cl'])

for i in range(len(list_fname_2022)):
    print(i)
    df = pd.read_csv(path_2022+list_fname_2022[i], sep=';')

    df['Quantity'] = df['Quantity'] * df['znak']
    df['BaseSum'] = df['BaseSum'] * df['znak']

    df = df[df['id_tt_cl'] == 11127].groupby(['id_tov_cl'], as_index=False)\
         .agg({'Quantity': 'sum', 'BaseSum':'sum'})\
         .rename(columns={'Quantity':f'Кол_{list_month[i]}', 'BaseSum':f'Сумма_{list_month[i]}'})

    df_prod_2022 = df_prod_2022.merge(df, how='outer', on='id_tov_cl')

df_prod_2022 = df_prod_2022.fillna(0)

df_prod_2022['Кол_345'] = (df_prod_2022['Кол_март'] + df_prod_2022['Кол_апр'] + df_prod_2022['Кол_май'])

df_prod_2022['Сумма_345'] = (df_prod_2022['Сумма_март'] + df_prod_2022['Сумма_апр'] + df_prod_2022['Сумма_май'])


0
1
2


In [18]:
df_finish = df_prod_2022.merge(gr_shop, how='right', on='id_tov_cl')


In [37]:
gr_tov = list(df_06_svao[df_06_svao['id_tt_cl']==11127].merge(name_tov, how='left', on='id_tov_cl')\
              .groupby('name_gr', as_index=False).agg({'name': pd.Series.nunique}).query('name > 2')['name_gr'])

In [38]:
gr_tov

['Айс ВкусВилл',
 'Алкоголь',
 'Бакалея',
 'Бытовая Химия',
 'Готовая еда',
 'Детские товары',
 'Детское питание',
 'Детское развитие, творчество и игры',
 'Замороженные десерты',
 'Замороженные продукты',
 'Консервация',
 'Косметика',
 'Кулинария',
 'Личная гигиена и уход',
 'Медицинские товары',
 'Мировые бренды',
 'Молочная Продукция',
 'Молочные продукты, яйцо',
 'Мясная гастрономия',
 'Мясные деликатесы. Колбасы',
 'Мясо, птица',
 'Мясо. Птица. П/ф охл. Яйца',
 'Напитки',
 'Овощи, фрукты, ягоды, зелень',
 'Овощи. Фрукты. Грибы. Зелень',
 'Рыба, икра и морепродукты',
 'Рыба. Морепродукты',
 'Сладости, десерты. Мороженое',
 'Сладости, кондитерские изделия',
 'Сухофрукты. Сушеные овощи, грибы, ягоды. Орехи',
 'Сыры',
 'Товары для дома',
 'Товары для животных',
 'Торты, пирожные, десерты',
 'Хлеб',
 'Хлеб и выпечка']

In [27]:
df_finish = df_finish[df_finish['name_gr'].isin(gr_tov)]

In [28]:
col = ['id_tov_cl', 'Кол_март', 'Сумма_март',
       'Кол_апр', 'Сумма_апр', 'Кол_май', 'Сумма_май', 'Кол_345', 'Сумма_345',
       'медиана кол', 'медиана сумма', 'кол/площ', 'кол/народ', 'сумма/площ',
       'сумма/народ', 'кол маг', 'список магазинов', 'name', 'name_gr2', 'name_gr',
       ]

In [29]:
df_finish[col].to_csv(path_2022+'11127/11127_s106_n2418_к_отбору.csv', sep=';', index=False)

In [30]:
df_finish

,id_tov_cl,Кол_март,Сумма_март,Кол_апр,Сумма_апр,Кол_май,Сумма_май,Кол_345,Сумма_345,медиана кол,медиана сумма,кол/площ,кол/народ,сумма/площ,сумма/народ,список магазинов,name,name_gr2,name_gr,кол маг
0,58738,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.50,313.00,0.44,0.76,92.24,158.09,"[15123, 14330]",Бешбармак с мясом говядины Айс,Вторые блюда мясо айс,Айс ВкусВилл,2
1,36785,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,8.00,1576.00,2.30,4.07,454.04,801.17,"[15123, 14330]",Биточки мясные с картофелем и грибами Айс,Вторые блюда мясо айс,Айс ВкусВилл,2
2,30513,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,168.00,0.38,0.35,64.63,59.29,[11439],Говядина в соусе с черносливом и гречкой Айс,Вторые блюда мясо айс,Айс ВкусВилл,1
3,29108,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3.00,586.00,1.15,1.16,225.45,227.01,"[11701, 15123, 14330, 11439, 11512]",Гречка с говядиной в сливочном соусе Айс,Вторые блюда мясо айс,Айс ВкусВилл,5
4,54599,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3.50,694.00,1.04,1.77,205.33,350.12,"[15123, 14330]",Котлета с паприкой и пюре из краснокочанной ка...,Вторые блюда мясо айс,Айс ВкусВилл,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2209,35775,13.00,496.36,18.00,677.55,3.00,114.55,34.00,1288.45,4.00,149.00,3.87,4.15,145.80,147.89,"[14343, 11912, 11457, 12337, 11312, 14988, 116...",Хлебцы с томатом и базиликом,"Хлебцы, сухарики, галеты, крекеры",Хлеб,13
2210,60978,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2.00,258.00,0.78,3.71,100.72,479.14,[11939],Хлеб для сэндвичей с коноплян. мукой без глюте...,Хлеб на закваске,Хлеб и выпечка,1
2211,64003,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2.00,86.00,1.01,1.27,44.20,55.51,"[11421, 12853, 11357, 11398, 11701, 15123, 120...",Палочки хлебные цельнозерновые,"Хлебцы, сухарики, галеты, крекеры",Хлеб и выпечка,20
2212,63437,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,77.00,0.90,0.95,69.00,68.98,"[10864, 12181, 11591, 12853, 11701, 14330, 117...",Сухарики ржаные с семечками и чесноком,"Хлебцы, сухарики, галеты, крекеры",Хлеб и выпечка,18
